# How this filter works
- 1. Assign `all_plugins.json` and to variable `all_plugins` to it
- 2. Create `blacklist_list` dict
  - 1. Get blacklist from `blacklist.json`
  - 2. Since `blacklist.json` is a nested dict, only get the leaf arrays and aggregate them into a single array. Set this array to `blacklist_list`
- 3. Assign `plugins_info.json` to variable `plugins_info`
- 4. Initiate a dict called inital_screening. Execute an initial screening of plugins with the following loop:
  - 1. Create a dict called `plugin_info` with the item `{namespace: plugin["namespace"], image: plugin["manifest"].get("logo_url", None), description_for_human: plugin["manifest"]["description_for_human"], description_for_model[plugin["manifest"]["description_for_model"]], openapi_url: plugin["manifest"]["api"]["url"]}`
  - 2. Check if `auth` is `"none"`
    - 1. If passes then add the following items to `plugin_info` to `{auth: false}`
    - 2. If fails then add the following items to `plugin_info` to `{auth: true}`
  - 3. Check that `namespace` is not it `blacklist_list`
    - 1. If passes then add the following items to `plugin_info` to `{blacklisted: false}`
    - 2. If fails then add the following items to `plugin_info` to `{blacklisted: true}`
  - 4. Add `plugin_info` to `plugins_info` with key of the namespace
- 5. Iterate through `plugins_info` and do the following:
  - 1. Check if `plugin_info["openapi_url"]` can be called without error
    - 1. If passes then add the following items to `plugin_info` to `{whitelisted: true}`
    - 2. If fails then add the following items to `plugin_info` to `{whitelisted: false}`
- 6. iterate through `openplugin_classes` to do a complete prompted test.
  - 1. generate a stimulous prompt to test the plugin
  - 2. use stimulous prompt to see if plugin can be called without error
- 7. Creaate a status for all plugins
  - 1. all plugins start with status `unsupported`
  - 2. if plugin `not openplugin_info["auth"] and not openplugin_info["blacklisted"] and openplugin_info["whitelisted"]` then status is `tentative`
  - 3. if plugin has `openplugin_info["stimulous_prompt"] and openplugin_info["stimulated"]` the status is `supported`
- 8. Create a file called `openplugins_compressed.json` and save only plugins that are `tentative` or `supported`
  - 1. Create a dict called `openplugins_compressed`
  - 2. Iterate through `plugins_info` and do the following:
    - 1. Check if `plugin_info["status"]` is `tentative` or `supported`
      - 1. If passes then add item {[namespace]: "https://" + openplugin_info["domain"]} to `openplugins_compressed`
- 9. create `PLUGINS.md` table based on `openplugins_compressed`

In [1]:
import json
import os
import re
import requests
from openplugincore import OpenPlugin
import openai
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

In [2]:
# handling isinstance_loop failure
with open('log.txt', 'r') as file:
    lines = file.read().splitlines()
    if lines:
        last_line = lines[-1]
        index_break, faulty_namespace = last_line.split()
        index_break = int(index_break)
    else:
        index_break, faulty_namespace = 1, None
if faulty_namespace:
    with open('blacklist.json', 'r') as file:
        blacklist = json.load(file)
    blacklist["isinstance_loop"].append(faulty_namespace)
    with open('blacklist.json', 'w') as file:
        json.dump(blacklist, file, indent=2)


In [3]:
# 1. Assign `all_plugins.json` to variable `all_plugins`
pypi_client_path = 'all_plugins.json'

with open(pypi_client_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Concatenate lines into a single JSON document
json_content = ''.join(lines)

try:
    all_plugins = json.loads(json_content)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

print(f"Number of plugins: {len(all_plugins)}\nFirst plugin: {json.dumps(all_plugins[0], indent=2)}")

Number of plugins: 615
First plugin: {
  "id": "plugin-985f0d2e-45f1-495c-b293-72ec96b1857f",
  "domain": "diary-chatgptplugin.derekxwang.com",
  "namespace": "diary",
  "status": "approved",
  "manifest": {
    "schema_version": "v1",
    "name_for_model": "diary",
    "name_for_human": "Diary",
    "description_for_model": "This plugin offers prompts for seamless diary management, coupled with a wealth of useful information related to maintaining a diary. It is particularly useful when users are working on their diaries and journals.",
    "description_for_human": "Kickstart your diary journey. Keeping a diary is designed to be effortless and beneficial.",
    "auth": {
      "type": "service_http",
      "instructions": "",
      "authorization_type": "bearer",
      "verification_tokens": {
        "openai": "6f4a6ab634264bda98df697b672b90c9"
      }
    },
    "api": {
      "type": "openapi",
      "url": "https://diary-chatgptplugin.derekxwang.com/.well-known/openapi.yaml"
    }

In [4]:
# 2. Create `blacklist_list` dict
with open('blacklist.json', 'r', encoding='utf-8') as f:
    blacklist = json.load(f)
def aggregate_items(json_object):
    aggregate_list = []
    for key in json_object:
        aggregate_list.extend(json_object[key])
    return aggregate_list
blacklist_list = aggregate_items(blacklist)
print(f"Number of blacklisted plugins: {len(blacklist_list)}\nBlacklist list: {blacklist_list}")

Number of blacklisted plugins: 33
Blacklist list: ['Coursera', 'video_insights', 'KAYAK', 'ArtCollection', 'AskMarcie', 'Avalara', 'BOXIL_SaaS', 'Broadway', 'BubbleGoods', 'ChatWithBible', 'ChatWithQuran', 'CryptoPrices', 'FreshTech', 'ABCmouse', 'staypia', 'DAIZY', 'Company_Transcripts', 'Bohita', 'CheckTheChain', 'Checkers', 'CloudDiagramGen', 'Diagrams', 'Creaticode_Extension_of_MIT_Scratch', 'Alchemy', 'ContentApp', 'Etihad_Airline', 'HaffPrice', 'JetBookClick', 'Product_Comparison', 'SEO', 'TestosteronBoostDiet', 'i_am_rich', 'universal']


In [5]:
# 3. Assign `plugins_info.json` to variable `plugins_info`
with open('openplugins_info.json', 'r', encoding='utf-8') as f:
    openplugins_info: dict = json.load(f)
first_openplugin_info = list(openplugins_info.keys())[0]

print(f"Number of plugins in openplugins_info: {len(openplugins_info)}\nFirst plugin in openplugins_info: {json.dumps(openplugins_info[first_openplugin_info], indent=2)}")

Number of plugins in openplugins_info: 609
First plugin in openplugins_info: {
  "auth": false,
  "blacklisted": true,
  "description_for_human": "Provides fun and educational learning activities for children 2-8 years old.",
  "description_for_model": "Assistant uses the ABCmouse plugin to get relevant learning activity suggestions for any child 2-8 years of age. Assistant will reply with the following 3 paragraphs 1) Activity Search Results [image] and [url] 2) Activity Learning Objectives [description] and [learning_objective] 3) Followup Questions. The first paragraph contains a list of the activities [url] with their learning attributes listed clearly and concisely as bullet points under the product [description], together with a link to the activity [url] and an explanation [learning_objective]. Links will always be returned and should be shown to the user. Assistant suggestions consider only the most important objectives of the activities [description, learning_objective] that w

In [10]:
# 4. Initiate a dict called inital_screening. Execute an initial screening of plugins with the following loop:
for plugin in all_plugins:
    openplugin_info = {
        **openplugins_info.get(plugin['namespace'], {}),
        'namespace': plugin['manifest']['name_for_model'],
        'image': plugin['manifest'].get('logo_url', None),
        'description_for_human': plugin['manifest'].get('description_for_human', None),
        'description_for_model': plugin['manifest'].get('description_for_model', None),
        'domain': plugin['domain'],
        'openapi_url': plugin['manifest'].get('api', {}).get('url', None)
    }
    if plugin['manifest'].get('auth', {}).get('type', None) == 'none':
        openplugin_info['auth'] = False
    else:
        openplugin_info['auth'] = True
    if plugin['namespace'] not in blacklist_list:
        openplugin_info['blacklisted'] = False
    else:
        openplugin_info['blacklisted'] = True
    openplugins_info[plugin['namespace']] = openplugin_info
with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"Number of plugins in openplugins_info: {len(openplugins_info)}\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")


Number of plugins in openplugins_info: 609
First openplugin info: {
  "auth": false,
  "blacklisted": true,
  "description_for_human": "Provides fun and educational learning activities for children 2-8 years old.",
  "description_for_model": "Assistant uses the ABCmouse plugin to get relevant learning activity suggestions for any child 2-8 years of age. Assistant will reply with the following 3 paragraphs 1) Activity Search Results [image] and [url] 2) Activity Learning Objectives [description] and [learning_objective] 3) Followup Questions. The first paragraph contains a list of the activities [url] with their learning attributes listed clearly and concisely as bullet points under the product [description], together with a link to the activity [url] and an explanation [learning_objective]. Links will always be returned and should be shown to the user. Assistant suggestions consider only the most important objectives of the activities [description, learning_objective] that will help th

In [11]:
# 5. Iterate through `plugins_info` and do the following:
openplugin_classes = {}
for idx, (namespace, openplugin_info) in enumerate(openplugins_info.items()):
    if openplugin_info["blacklisted"]:
        continue
    if idx < index_break - 2:
        continue
    with open('log.txt', 'a', encoding='utf-8') as f:
        f.write(f"{idx} {namespace}\n")
    if idx % 20 == 0:
        print(f"Processing plugin {idx} of {len(openplugins_info)}")
    # (namespace, plugin_info) = list(openplugins_info.items())[12]
    root_url = "https://" + openplugin_info['domain']
    try:
        openplugin_classes[namespace] = OpenPlugin(openai_api_key=OPENAI_API_KEY, root_url=root_url)
        print(idx, namespace, " success")
        openplugin_info['whitelisted'] = True
    except Exception as e:
        print(f"{idx} {namespace} Error: {e}")
        openplugin_info['whitelisted'] = False
    openplugins_info[namespace] = openplugin_info

with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]

whitelisted_count = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info['whitelisted'] == True:
        whitelisted_count += 1
print(f"{whitelisted_count} whitelisted plugins of out of {len(openplugins_info)}\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


1 AFinChat  success


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


2 AI2sql  success


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


...

606 wpinteract  success


Attempting to load an OpenAPI 3.0.3 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


607 yabble  success


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


608 yt_caption_retriever  success
494 whitelisted plugins of out of 609
First openplugin info: {
  "auth": false,
  "blacklisted": true,
  "description_for_human": "Provides fun and educational learning activities for children 2-8 years old.",
  "description_for_model": "Assistant uses the ABCmouse plugin to get relevant learning activity suggestions for any child 2-8 years of age. Assistant will reply with the following 3 paragraphs 1) Activity Search Results [image] and [url] 2) Activity Learning Objectives [description] and [learning_objective] 3) Followup Questions. The first paragraph contains a list of the activities [url] with their learning attributes listed clearly and concisely as bullet points under the product [description], together with a link to the activity [url] and an explanation [learning_objective]. Links will always be returned and should be shown to the user. Assistant suggestions consider only the most important objectives of the activities [description, learning

In [15]:
# 6. iterate through `openplugin_classes`
print(f"Number of openplugin_classes: {len(openplugin_classes)}")
for idx, (namespace, openplugin_class) in enumerate(openplugin_classes.items()):
    # namespace, openplugin_class = "Ai_PDF", OpenPlugin("", root_url="https://" + openplugins_info["Ai_PDF"]["domain"])
    openplugin_info = openplugins_info[namespace]
    openplugin_info["stimulous_prompt"] = openplugin_info.get("stimulous_prompt", None)
    openplugin_info["stimulated"] = openplugin_info.get("stimulated", False)
    if not openplugin_info["auth"] and not openplugin_info["blacklisted"] and openplugin_info["whitelisted"]:
        try:
            if openplugin_info.get("stimulous_prompt", None) is None:
                generate_stimulation_prompt_prompt = {
                    "prompt": f"""
                    Please create a prompt that will trigger an model's plugin with the human description delimited by driple backticks.
                    If necessary also look at the model description also delimited by triple backticks.
                    Please do not ask anything from the AI you should provide all the information it needs in the prompt.
                    You should not be ambiguous or open ended in your prompt use specific examples.
                    Do not simply restate the description.
                    Human description:
                    ```
                    {openplugin_info["description_for_human"]}
                    ```
                    Model description:
                    ```
                    {openplugin_info["description_for_model"]}
                    ```
                    """,
                    "function": {
                    "name": "stimulous_prompt_generation",
                    "description": """
                    Generates a natural language phrase to that triggeres the AI plugin.
                    If approriate the phrase should include an example item/url (https://github.com/)/text/ect. even if you are not sure if it is real its ok to make it up.
                    """,
                    "parameters": {
                        "type": "object",
                        "properties": {
                        "stimulous_prompt": {
                            "type": "string",
                            "description": "The stimulous phrase to trigger the AI plugin"
                        },
                        },
                        "required": ["stimulous_prompt"]
                    }
                    }
                }
                generation = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo-0613",
                    temperature=0,
                    messages=[{"role": "user", "content": generate_stimulation_prompt_prompt["prompt"]}],
                    functions=[generate_stimulation_prompt_prompt["function"]],
                    function_call={"name": "stimulous_prompt_generation"}
                )
                json_arguments = json.loads(generation["choices"][0]["message"]["function_call"]["arguments"])
                openplugin_info["stimulous_prompt"] = json_arguments["stimulous_prompt"]
            function_response = openplugin_class.fetch_plugin(
                prompt=openplugin_info["stimulous_prompt"],
                model="gpt-3.5-turbo-0613",
                temperature=0,
            )
            openplugin_info["stimulated"] = True
            print(f"{idx} Plugin '{namespace}' stimulated with prompt:\n{openplugin_info['stimulous_prompt']}")
        except Exception as e:
            openplugin_info["stimulated"] = False
            print(f"{idx} Plugin '{namespace}' failed with error:\n{e}")

with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)

num_stimulous_prompt = 0
num_stimulated = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["stimulous_prompt"]:
        num_stimulous_prompt += 1
    if openplugin_info["stimulated"]:
        num_stimulated += 1
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"{num_stimulous_prompt} plugins with stimulous prompts, {num_stimulated} plugins stimulated, out of {len(openplugins_info)} plugins\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")

Number of openplugin_classes: 462
2 Plugin 'AIMaster' failed with error:
Invalid URL '//query/prompt/psychologist': No scheme supplied. Perhaps you meant https:////query/prompt/psychologist?
4 Plugin 'AMAZON_product_desc_generator' failed with error:
Not a plugin function
7 Plugin 'Agones' failed with error:
Not a plugin function
8 Plugin 'Ai_PDF' stimulated with prompt:
Please summarize the following PDF https://www.unodc.org/pdf/criminal_justice/UN_Basic_Principles_on_the_Role_of_Lawyers.pdf
9 Plugin 'Algorithma' failed with error:
Invalid URL '//start': No scheme supplied. Perhaps you meant https:////start?
11 Plugin 'ApexMap' failed with error:
Expecting value: line 1 column 1 (char 0)
12 Plugin 'AppyPieAIAppBuilder' stimulated with prompt:
Generate an Android and iOS app using the AI-powered Text-to-App Generator. Just provide the following text input:

```
AI-powered Text-to-App Generator turns your app idea into Android and iOS apps- just provide text input.
```
13 Plugin 'AskCa

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 06 Jul 2023 02:38:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e2471ef6e29e174-ORD', 'alt-svc': 'h3=":443"; ma=86400'}.


157 Plugin 'Weather_and_Train_and_in_Korea' failed with error:
Invalid URL '//weather_now/Seoul': No scheme supplied. Perhaps you meant https:////weather_now/Seoul?
158 Plugin 'WebRewind' stimulated with prompt:
Get the picture of the website `https://github.com/` on `January 1, 2022`. Provide the image in Markdown format and give a brief description of the website at that time.
160 Plugin 'WebsiteChat' failed with error:
Invalid URL '//loadSite': No scheme supplied. Perhaps you meant https:////loadSite?
162 Plugin 'WordCloud' stimulated with prompt:
Generate a word cloud image from the following text: ```The quick brown fox jumps over the lazy dog.```
163 Plugin 'XWeather' failed with error:
Invalid URL '//weather/summary/New York City': No scheme supplied. Perhaps you meant https:////weather/summary/New York City?
164 Plugin 'Xpapers_arXiv_paper_database' stimulated with prompt:
Search for academic papers on arXiv using the plugin. Find papers related to 'machine learning' and access

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 06 Jul 2023 02:47:16 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e247df86ae1118b-ORD', 'alt-svc': 'h3=":443"; ma=86400'}.


220 Plugin 'coincap' failed with error:
Not a plugin function
221 Plugin 'companieshouse' failed with error:
Not a plugin function
222 Plugin 'competitorppcads' failed with error:
Not a plugin function
223 Plugin 'converter_app' failed with error:
Expecting value: line 1 column 1 (char 0)
224 Plugin 'copilot' failed with error:
'get_vehicles_for_sale_by_coordinates_vehicles_for_sale_by_coordinates_get' does not match '^[a-zA-Z0-9_-]{1,64}$' - 'functions.4.name'
225 Plugin 'coupons_by_tenereteam' failed with error:
Expecting property name enclosed in double quotes: line 3 column 1 (char 253)
226 Plugin 'crafty_clues' stimulated with prompt:
Play a game of Crafty Clues (a word guessing game) with the user. Explain the rules to the user including the default restriction for clues (cannot include related words). Ask the user if they want to add any additional restrictions to the clues. Tell them that they can also mix and match restrictions or come up with their own to make the game more i

...

336 Plugin 'nba_stats' stimulated with prompt:
Retrieve NBA stats for the 2020-2021 season. Analyze the performance of the Los Angeles Lakers and their players in the NBA Finals. Provide insights on the shooting percentages, rebounds, assists, and turnovers of LeBron James and Anthony Davis. Compare their performance to the overall team statistics. Use the basketball stats plugin to gather the required data.
337 Plugin 'ndricks_sports_api' stimulated with prompt:
Retrieve information about the NHL teams using the ndricks Software Sports API.
338 Plugin 'netools' failed with error:
None is not of type 'string' - 'functions.1.description'
339 Plugin 'nextpaper' stimulated with prompt:
Fetch the latest research papers on a specific topic from PubMed. For example, retrieve the latest paper on COVID-19. More to come.
340 Plugin 'nftguru' failed with error:
Expecting value: line 1 column 1 (char 0)
341 Plugin 'nonprofits' failed with error:
Not a plugin function
344 Plugin 'openarch' stimula

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


348 Plugin 'passiveDNS' failed with error:
Invalid URL '//api/chatGPT/qname': No scheme supplied. Perhaps you meant https:////api/chatGPT/qname?
349 Plugin 'paxiai' stimulated with prompt:
As a human, I want to find the right AI tool for a specific task, so that I can complete my tasks more efficiently. For example, I need an AI tool to analyze customer feedback and provide sentiment analysis.
351 Plugin 'photorealistic' stimulated with prompt:
Generate a photorealistic prompt for the Midjourney image creation tool. For example, create a prompt for generating a realistic image of a beach sunset.
353 Plugin 'plooral' stimulated with prompt:
Search for a job as a software engineer on the Plooral Eduployment platform.
354 Plugin 'podcastSearch' failed with error:
'podcasttitle'
355 Plugin 'polarr' failed with error:
Session.request() got an unexpected keyword argument 'prompt'
356 Plugin 'pollthepeople' failed with error:
Not a plugin function
357 Plugin 'portfoliopilot' stimulated with p

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Thu, 06 Jul 2023 03:05:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e249811acc510df-ORD', 'alt-svc': 'h3=":443"; ma=86400'}.


442 Plugin 'weather_meteosource' stimulated with prompt:
Get the current weather and forecast for New York City.
443 Plugin 'web5' failed with error:
Not a plugin function
445 Plugin 'web_pilot' stimulated with prompt:
Generate an article from the URL 'https://github.com/' and extract specific information about the webpage.
446 Plugin 'web_requests' failed with error:
Invalid URL '//scrape_url': No scheme supplied. Perhaps you meant https:////scrape_url?
449 Plugin 'welt_news_verse' stimulated with prompt:
Stay informed with the latest news from welt.de.
450 Plugin 'what_to_watch' failed with error:
Not a plugin function
451 Plugin 'whimsical' stimulated with prompt:
Create a flowchart with the following diagram:

graph TD
  A[Start] --Connection--> B[End]

The title of the diagram should be 'Flowchart Example'.

After creating the diagram, render it as an inline image and display the link to edit the diagram in Whimsical below the image. The link text should be 'View or edit this diag

In [16]:
# 7. Creaate a status for all plugins
for namespace, openplugin_info in openplugins_info.items():
    openplugin_info["status"] = "unsupported"
    if not openplugin_info["auth"] and not openplugin_info["blacklisted"] and openplugin_info["whitelisted"]:
        openplugin_info["status"] = "tentative"
    if openplugin_info["stimulous_prompt"] and openplugin_info["stimulated"]:
        openplugin_info["status"] = "supported"
    openplugins_info[namespace] = openplugin_info
with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)

num_unsupported = 0
num_tentative = 0
num_supported = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["status"] == "unsupported":
        num_unsupported += 1
    if openplugin_info["status"] == "tentative":
        num_tentative += 1
    if openplugin_info["status"] == "supported":
        num_supported += 1
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"{num_unsupported} plugins with status unsupported, {num_tentative} plugins with status tentative, {num_supported} plugins with status supported, out of {len(openplugins_info)} plugins\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")

271 plugins with status unsupported, 177 plugins with status tentative, 161 plugins with status supported, out of 609 plugins
First openplugin info: {
  "auth": false,
  "blacklisted": true,
  "description_for_human": "Provides fun and educational learning activities for children 2-8 years old.",
  "description_for_model": "Assistant uses the ABCmouse plugin to get relevant learning activity suggestions for any child 2-8 years of age. Assistant will reply with the following 3 paragraphs 1) Activity Search Results [image] and [url] 2) Activity Learning Objectives [description] and [learning_objective] 3) Followup Questions. The first paragraph contains a list of the activities [url] with their learning attributes listed clearly and concisely as bullet points under the product [description], together with a link to the activity [url] and an explanation [learning_objective]. Links will always be returned and should be shown to the user. Assistant suggestions consider only the most importa

In [17]:
# 8. Create a file called `openplugins_compressed.json` and save only plugins that are `tentative` or `supported`
openplugins = {"__testing__": "http://localhost:3333"} # default testing url
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["status"] == "tentative" or openplugin_info["status"] == "supported":
        openplugins[namespace] = "https://" + openplugin_info["domain"]

with open('openplugins.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins, f, indent=2, sort_keys=True)

first_openplugin_compressed = openplugins[list(openplugins.keys())[0]]
print(f"{len(openplugins)} plugins in openplugins_compressed\nFirst openplugin compressed: {json.dumps(first_openplugin_compressed, indent=2)}")

338 plugins in openplugins_compressed
First openplugin compressed: "https://plugin.chat2any.com"


In [6]:
# 9. create PLUGINS.md table
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

# Create the file path for plugins.md
pluginsmd_path = os.path.join(root_dir, 'PLUGINS.md')
plugins_md = """
# Plugins
Available plugins for OpenPlugin
Status:
- `tentative`: passed basic tests (may work)
- `supported`: passed complete prompt tests (should work)

| Image | Namespace | Status | Description | Description for model |
| --- | --- | --- | --- | --- |
"""

def escape_special_markdown_chars(text):
    # Characters to escape: \ ` * _ { } [ ] ( ) # + !
    special_chars = r'\\|`|\*|_|{|}|\[|\]|\(|\)|#|\+|!'
    return re.sub(special_chars, lambda match: '\\' + match.group(), text)

def remove_line_breaks(text):
    return text.replace('\n', ' ').replace('\r', '')

# create two lists of plugins one supported_plugins and one tentative_plugins
supported_plugins = []
tentative_plugins = []
for _namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["status"] == "tentative":
        tentative_plugins.append(openplugin_info)
    if openplugin_info["status"] == "supported":
        supported_plugins.append(openplugin_info)
# now sort the lists by their namespace keys considering that each plugin is a dict of {namespace: str, ...}
supported_plugins.sort(key=lambda x: x["namespace"])
tentative_plugins.sort(key=lambda x: x["namespace"])
# aggragate the lists so that supported_plugins is first and tentative_plugins is second 
ordered_openplugins_info = supported_plugins + tentative_plugins

for openplugin_info in ordered_openplugins_info:
        if openplugin_info["image"]:
            image = escape_special_markdown_chars(openplugin_info["image"])
        else:
            image = escape_special_markdown_chars("https://i.imgur.com/L3giCRt.png")
        namespace = escape_special_markdown_chars(openplugin_info["namespace"])
        status = escape_special_markdown_chars(openplugin_info["status"])
        description = escape_special_markdown_chars(remove_line_breaks(remove_line_breaks(openplugin_info["description_for_human"])))
        description_for_model = escape_special_markdown_chars(remove_line_breaks(openplugin_info["description_for_model"]))
        plugins_md += f"| ![{namespace} Logo]({image}) | {namespace} | {status} | {description} | {description_for_model} |\n"

with open(pluginsmd_path, 'w', encoding='utf-8') as f:
    f.write(plugins_md)

print(f"Successfully created PLUGINS.md file with {len(ordered_openplugins_info)} plugins of {len(openplugins_info)} total plugins.")

Successfully created PLUGINS.md file with 338 plugins of 609 total plugins.
